In [5]:
import sys
sys.path.append('../metanas')

from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from metanas.tasks.omniprint import omniprint
from torchmeta.datasets import Omniglot
from torchmeta.transforms import Categorical, ClassSplitter, Rotation
from torchvision.transforms import Compose, Resize, ToTensor, Grayscale
from torchmeta.utils.data import BatchMetaDataLoader
from torchmeta.datasets.helpers import miniimagenet, triplemnist

from metanas.tasks.core import TaskDistribution, Task

In [38]:

def sample_meta_batch(
    batch_iter, meta_batch_size, task_batch_size, shots, ways,
    task_train_sampler=None
):
    """Sample a meta batch using a torchmeta :class:`BatchMetaDataLoader`

    Args:
        batch_iter: Iterator for the :class:`BatchMetaDataLoader`
        meta_batch_size: Number of tasks per meta-batch
        task_batch_size: Number of elements in a task batch
        shots: Number of samples per class
        ways: Number of classes per task
        task_train_sampler: Samples for the meta batch train dataset

    Returns:
        A list of data loaders for training, validation and testing for one
        task. Currently, the validation loader is the same as the training
        loader.
    """
    batch = next(batch_iter)
    train_batch_x, train_batch_y = batch["train"]
    test_batch_x, test_batch_y = batch["test"]
    num_tasks = meta_batch_size

    meta_train_batch = list()
    for task_idx in range(num_tasks):
        dset_train = TensorDataset(
            train_batch_x[task_idx], train_batch_y[task_idx])
        dset_val = TensorDataset(
            test_batch_x[task_idx], test_batch_y[task_idx])
        train_loader = DataLoader(
            dset_train, batch_size=task_batch_size, sampler=task_train_sampler
        )
        test_loader = DataLoader(dset_val, batch_size=shots * ways)
        meta_train_batch.append(Task(train_loader, train_loader, test_loader))
    return meta_train_batch


In [80]:
dataset = omniprint(
    "/home/rob/Desktop",
    1,
    20,
    print_split='meta1',
    meta_split='test',
    test_shots=1,
    download=True,
    seed=1,
)
dataloader = BatchMetaDataLoader(
    dataset, batch_size=10, num_workers=1, shuffle=True
)
test_iter = iter(dataloader)

In [81]:
task = sample_meta_batch(
    test_iter, 10, 10, 1, 20,
    task_train_sampler=None)

In [82]:
k = next(iter(task.train_loader))

AttributeError: 'list' object has no attribute 'train_loader'

In [83]:
k[0].shape

torch.Size([20, 1, 32, 32])

In [74]:
task[0].train_loader

In [84]:

for step, (train_X, train_y) in enumerate(task[0].train_loader):
    print(train_X.shape)
    print(step*10)

torch.Size([10, 1, 32, 32])
0
torch.Size([10, 1, 32, 32])
10


In [88]:
import torch

In [89]:
train_X.shape

torch.Size([10, 1, 32, 32])

In [90]:
torch.tile(train_X, (20, 1, 32, 32))

AttributeError: module 'torch' has no attribute 'tile'

In [91]:
train_X.tile((20, 1, 32, 32))

AttributeError: 'Tensor' object has no attribute 'tile'

In [92]:
torch.__version__

'1.6.0'

In [94]:
train_X.repeat(2, 1, 1, 1).shape

torch.Size([20, 1, 32, 32])

In [95]:
x = torch.tensor([1, 2, 3])
x

tensor([1, 2, 3])

In [99]:
x.repeat(4, 1, 1)

tensor([[[1, 2, 3]],

        [[1, 2, 3]],

        [[1, 2, 3]],

        [[1, 2, 3]]])

In [151]:
import copy


def fill_up_dataset(dataset, required_size=None):
    ds_size = x.shape[0]
    ds = copy.deepcopy(dataset)
    
    # In case only downsizing is required
    if ds_size >= required_size:
        return dataset[:required_size]

    while ds.size(0) < required_size:
        perm = torch.randperm(len(ds))[:1].item()
        ds = torch.cat((ds, ds[perm].unsqueeze(dim=0)))
    return ds

In [153]:
fill_up_dataset(train_X, required_size=20).shape

torch.Size([20, 1, 32, 32])

In [ ]:
perm = torch.randperm(1).item()

In [135]:
import copy
X = copy.deepcopy(train_X)

In [148]:
required_size = 20    

while X.size(0) < required_size:
    perm = torch.randperm(len(X))[:1].item()
    X = torch.cat((X, X[perm].unsqueeze(dim=0)))
    print(X.shape)

0
torch.Size([11, 1, 32, 32])
3
torch.Size([12, 1, 32, 32])
4
torch.Size([13, 1, 32, 32])
8
torch.Size([14, 1, 32, 32])
1
torch.Size([15, 1, 32, 32])
11
torch.Size([16, 1, 32, 32])
6
torch.Size([17, 1, 32, 32])
5
torch.Size([18, 1, 32, 32])
1
torch.Size([19, 1, 32, 32])
17
torch.Size([20, 1, 32, 32])


In [129]:
X[perm].unsqueeze(dim=0).shape

torch.Size([1, 1, 32, 32])

In [130]:
perm = torch.randperm(1).item()
torch.cat((X, X[perm].unsqueeze(dim=0))).shape

torch.Size([11, 1, 32, 32])

In [132]:
X.shape

torch.Size([10, 1, 32, 32])

In [142]:
torch.randperm(8).item()

ValueError: only one element tensors can be converted to Python scalars

In [109]:
train_X.shape

torch.Size([10, 1, 32, 32])

In [111]:
train_X[:10].shape

torch.Size([10, 1, 32, 32])

In [ ]:
ds_base = copy.deepcopy(dataset)

In [147]:
torch.randperm(len(X))[:1].item()

2

In [1]:
import pickle

In [6]:
k = open("/home/rob/Git/meta-fsl-nas/metanas/results/omniglot_n1_k20/ppo_metad2a_env_1/seed_2/_s2/vars10.pkl", 'rb')

m = pickle.load(k)

In [7]:
m

In [9]:
import torch


a=torch.zeros(20,1,32,32)

In [11]:
torch.split(a, 10)[0].shape

torch.Size([10, 1, 32, 32])

In [ ]:

normal_cells = list()
red_cells = list()
for lyr, cell in enumerate(self.net.cells):
    if cell.reduction:
        red_cells.append(lyr)
    else:
        normal_cells.append(lyr)

In [ ]:


for layer_name, layer_weights in self.named_parameters():
    # check if layer is part of normal or reduction cell
    if "net.cells." in layer_name:  # layer part of cells at all?
        for cell in normal_cells:  # normal cell?
            if "net.cells." + str(cell) in layer_name:  # normal cell
                none_active_ops = none_active_ops_normal

        # else reduction cell
        for cell in red_cells:
            if "net.cells." + str(cell) in layer_name:  # normal cell
                none_active_ops = none_active_ops_reduce

        if any(
            [none_active_op in layer_name for none_active_op in
                none_active_ops]
        ):  # check if layer is part of none-active ops
            none_active_params += layer_weights.numel()

active_params = all_params - none_active_params

In [22]:
import sys
sys.path.append('../metanas')

from metanas.utils.test_config import init_config
from metanas.models.search_cnn import SearchCNNController

In [25]:
config = init_config()

model = SearchCNNController(
    28,
    config.init_channels,
    20,
    config.layers,
    n_nodes=config.nodes,
    reduction_layers=config.reduction_layers,
    device_ids=config.gpus,
#     normalizer=normalizer,
    PRIMITIVES=config.primitives,
    feature_scale_rate=1,
    primitive_space=config.primitives_type,
    weight_regularization=config.darts_regularization,
    use_hierarchical_alphas=config.use_hierarchical_alphas,
    use_pairwise_input_alphas=config.use_pairwise_input_alphas,
    alpha_prune_threshold=config.alpha_prune_threshold,  
)

Using pairwise input alphas.
Using dropout on skip-connections False
Reduction layers: [1, 3]


In [ ]:

normal_cells = list()
red_cells = list()
for lyr, cell in enumerate(self.net.cells):
    if cell.reduction:
        red_cells.append(lyr)
    else:
        normal_cells.append(lyr)

In [78]:
next(model.parameters())

Parameter containing:
tensor([[-0.0003,  0.0001,  0.0005, -0.0006, -0.0015, -0.0002, -0.0009],
        [ 0.0013, -0.0016,  0.0007,  0.0016,  0.0001, -0.0011, -0.0008]],
       requires_grad=True)

In [90]:
# def k():

def p():
    j = list()
    for lyr, cell in enumerate(model.net.cells):
    #     print(lyr, _) #cell
    #     print(cell)
        if cell.reduction:
            p = cell.parameters()
            j.extend(list(p))
    return j
    #         print(lyr, cell.parameters())
    #             yield p

In [100]:

for n, p in model.named_parameters():
    if "alpha_normal" in n:
        print(n, p)
    #     if "alpha" in n:
#         self._alphas.append((n, p))

alpha_normal.0 Parameter containing:
tensor([[-0.0003,  0.0001,  0.0005, -0.0006, -0.0015, -0.0002, -0.0009],
        [ 0.0013, -0.0016,  0.0007,  0.0016,  0.0001, -0.0011, -0.0008]],
       requires_grad=True)
alpha_normal.1 Parameter containing:
tensor([[-1.1244e-03, -1.3794e-03,  8.3177e-04,  9.7086e-04, -2.3417e-04,
          1.9822e-03, -2.0120e-03],
        [-1.3346e-03, -4.0410e-04, -3.6764e-04,  9.4293e-04, -1.5030e-03,
         -6.9980e-04, -2.9795e-03],
        [ 7.5656e-04,  7.8027e-04, -8.1603e-05, -1.5209e-04,  4.8610e-04,
          7.4281e-04,  2.7059e-03]], requires_grad=True)
alpha_normal.2 Parameter containing:
tensor([[-1.1350e-03, -2.0869e-03,  8.3288e-04, -1.6022e-03,  9.8372e-05,
          8.1471e-04,  7.5671e-04],
        [-1.1453e-03, -4.8925e-04, -3.5947e-04, -7.3000e-04, -2.1004e-04,
         -1.8787e-03, -5.2337e-04],
        [-5.2203e-04,  1.2958e-03,  1.7968e-03, -3.8278e-04, -5.7638e-04,
          1.7829e-03, -4.7939e-04],
        [ 9.0611e-04,  4.5175e-04,

In [92]:

# cell.parameters()

In [93]:

torch.optim.Adam(
    p(),
    0.01,
)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)

In [7]:
import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, DistributedSampler
from torchvision.datasets import FashionMNIST


class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(500, 10)
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x, **kwargs):
        batch_size = x.shape[0]
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(batch_size, -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x


def test(model, data_loader):
    print("Test started...")
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in data_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: %d %%' % (
            100 * correct / total))


def train(model, data_loader):
    optimizer = optim.Adam(model.parameters())
    criterion = nn.NLLLoss()

    for data, labels in tqdm.tqdm(data_loader):
        optimizer.zero_grad()
        loss = criterion(model(data), labels)
        loss.backward()        
        optimizer.step()
        break
    return "done"


if __name__ == '__main__':
    num_processes = 4
    model = Model()
    model.share_memory()

    dataset = CIFAR10(
                "data",
                train=True,
                download=True,
                transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(
                        mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225]
                    )
                ])
            )

    testset = CIFAR10(
        "data",
        train=False,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
    )

    processes = []
    for rank in range(num_processes):
        data_loader = DataLoader(
            dataset=dataset,
            sampler=DistributedSampler(
                dataset=dataset,
                num_replicas=num_processes,
                rank=rank
            ),
            batch_size=32
        )
        p = mp.Process(target=train, args=(model, data_loader))
        p.start()
        processes.append(p)
    for p in processes:
        p.join()

    test(model, DataLoader(
        dataset=testset,
        batch_size=1000
    ))

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/391 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F


def train(rank, model, device, dataset, dataloader_kwargs):
    torch.manual_seed(1 + rank)

    train_loader = torch.utils.data.DataLoader(dataset, **dataloader_kwargs)

    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
    for epoch in range(1, args.epochs + 1):
        train_epoch(epoch, args, model, device, train_loader, optimizer)


def test(args, model, device, dataset, dataloader_kwargs):
    torch.manual_seed(args.seed)

    test_loader = torch.utils.data.DataLoader(dataset, **dataloader_kwargs)

    test_epoch(model, device, test_loader)


def train_epoch(epoch, args, model, device, data_loader, optimizer):
    model.train()
    pid = os.getpid()
    for batch_idx, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = F.nll_loss(output, target.to(device))
        loss.backward()
        optimizer.step()
        print("yo")
        if batch_idx % args.log_interval == 0:
            print('{}\tTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                pid, epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item()))
            if args.dry_run:
                break


def test_epoch(model, device, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            output = model(data.to(device))
            test_loss += F.nll_loss(output, target.to(device), reduction='sum').item() # sum up batch loss
            pred = output.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target.to(device)).sum().item()

    test_loss /= len(data_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

In [16]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
from torch.utils.data.sampler import Sampler
from torchvision import datasets, transforms

# from train import train, test

# Training settings
# parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
# parser.add_argument('--batch-size', type=int, default=64, metavar='N',
#                     help='input batch size for training (default: 64)')
# parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
#                     help='input batch size for testing (default: 1000)')
# parser.add_argument('--epochs', type=int, default=10, metavar='N',
#                     help='number of epochs to train (default: 10)')
# parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
#                     help='learning rate (default: 0.01)')
# parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
#                     help='SGD momentum (default: 0.5)')
# parser.add_argument('--seed', type=int, default=1, metavar='S',
#                     help='random seed (default: 1)')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='how many batches to wait before logging training status')
# parser.add_argument('--num-processes', type=int, default=2, metavar='N',
#                     help='how many training processes to use (default: 2)')
# parser.add_argument('--cuda', action='store_true', default=False,
#                     help='enables CUDA training')
# parser.add_argument('--dry-run', action='store_true', default=False,
#                     help='quickly check a single pass')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


if __name__ == '__main__':
#     args = parser.parse_args()

    use_cuda = True #args.cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    kwargs = {'batch_size': 32,
              'shuffle': True}
    if use_cuda:
        kwargs.update({'num_workers': 1,
                       'pin_memory': True,
                      })

    torch.manual_seed(1)
#     mp.set_start_method('spawn')

    model = Net().to(device)
    model.share_memory() # gradients are allocated lazily, so they are not shared here

    processes = []
    for rank in range(2):
        p = mp.Process(target=train, args=(rank, model, device,
                                           dataset1, kwargs))
        # We first train the model across `num_processes` processes
        p.start()
        processes.append(p)
    for p in processes:
        p.join()

    # Once training is complete, we can test the model
    test(args, model, device, dataset2, kwargs)

NameError: name 'args' is not defined

In [2]:
import gym

In [6]:
env = gym.make("CartPole-v1")
env.action_space.n

2

In [7]:
from collections import namedtuple


TrialFinished = namedtuple('TrialFinished', ['agent', 'finished'])

In [9]:
TrialFinished.__name__

'EpisodeFinished'

In [12]:
import torch
import numpy as np

In [26]:
a2 = np.array([0, 0])
r2 = np.array([0, 0])


prev_act = torch.as_tensor(
    a2, dtype=torch.float32
)#.squeeze(0)

# rew shape: [2]
prev_rew = torch.as_tensor(
    r2, dtype=torch.float32
).unsqueeze(1)

# prev rew, [2, 1]
prev_act.shape, prev_rew.shape

(torch.Size([2]), torch.Size([2, 1]))

In [32]:
np.zeros(2)

array([0., 0.])

In [28]:
torch.rand(2, 100).unsqueeze(0).shape

torch.Size([1, 2, 100])

In [30]:
np.zeros((self.num_workers,))

NameError: name 'self' is not defined

In [49]:
import torch
import numpy as np

In [55]:
samples = torch.rand(200, 500)
samples.shape

torch.Size([200, 500])

In [62]:
indices = torch.arange(0, 200 * 500).reshape(200, 500)
indices.shape

batch_indices = indices[:50]
sequence_indices = torch.randperm(500)

batch_indices.shape, sequence_indices.shape

(torch.Size([50, 500]), torch.Size([500]))

In [60]:
# num_sequences_per_batch = [50] * 4
# remainder = 200 % 50
num_sequences_per_batch = 50

In [61]:
np.random.choice(sequence_indices.numpy(), size=10)

array([447, 430, 251, 153, 281, 274, 309,  50, 150, 282])

In [65]:
x = torch.ones((5, 16))
x

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [67]:
idx = [1, 2]

x[idx] = torch.zeros(16)

In [68]:
x

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])